# Initial Model Testing

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.preprocessing import MultiLabelBinarizer

print(tf.__version__)

1.12.0


In [2]:
def tokenize_and_pad(test_sentence):
    test_sentence = tokenizer.texts_to_sequences([test_sentence])
    test_sentence = sequence.pad_sequences(test_sentence, maxlen=max_length, padding="post")
    return test_sentence

In [3]:
from tensorflow.keras.preprocessing import text, sequence
import pickle

with open('tokenizer_imdb.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    
max_length = 2722
vocab_size = len(tokenizer.word_index) + 1

# test_sentence = ['Sapura Energy bags 5 new contracts worth RM1.3 billion - Free Malaysia Today', 
#                  'BREAKING: Trump just signed two executive orders that seek to expedite permits for pipelines and other fossil fuel projects by restricting public input and states authority.',
#                 'Number of companies producing oil and gas in Western Canada drops 17.5% since 2014', 
#                 'Wheelchair customers stuck out in the pouring rain when the cab was booked in advance is OUTRAGEOUS. This cab is now 55 minutes late and my client has missed a VERY important DRs appt. BRING #Uber TO VANCOUVER if anything just to SCREW with @vancouvertaxi customers. @NEWS1130',
#                 'Uber worst service... Cab booked to take 1.5 year child to doctor and cab did not come more then 1hr driver not answering and not cancelling the ride...',
#                 'That #Uber ad sums up that company!! I’ve no sympathy for anyone male or female who uses em and becomes unstuck! They’re not even fucking cheap that’s a myth! Use your local mini cab firm n black cabs in London! Let’s keep money in England ay n drive this firm out!',
#                 'Uber is awesome!',
#                 'Looking forward to your  presentation at Benzinga next week! Armenians represent! ']


test_sentence = tokenize_and_pad('Sapura Energy bags 5 new contracts worth RM1.3 billion - Free Malaysia Today')


# test_sentence = tokenizer.texts_to_sequences(test_sentence)
# test_sentence = sequence.pad_sequences(test_sentence, maxlen=max_length, padding="post")
test_sentence

array([[1699, 8860,  451, ...,    0,    0,    0]], dtype=int32)

# Load Model

In [5]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate, Flatten, Embedding
from tensorflow.keras.layers import Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, Input, LSTM, CuDNNLSTM
# from tensorflow.compat.v1.keras.layers import CuDNNLSTM, CuDNNGRU

## Simple LSTM Eager

In [18]:
BATCH_SIZE = 64
EMBEDDING_DIM = 500
LSTM_UNITS = 256
DENSE_UNITS = LSTM_UNITS * 4

class MyLSTM(tf.keras.Model):
    def __init__(self):
        super(MyLSTM, self).__init__()
        self.embedding = Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length)
        self.dropout1 = SpatialDropout1D(0.3)
        self.lstm1 = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True, dropout=0.2))
        self.gmp = GlobalMaxPooling1D()
        self.dense1 = Dense(100, activation='relu')
        self.dropout2 = Dropout(0.2)
        self.denseOut = Dense(1, activation='sigmoid')
    
    def call(self, x):
        x = self.embedding(x)
        x = self.dropout1(x)
        x = self.lstm1(x)
        x = self.gmp(x)
        x = self.dense1(x)
        x = self.dropout2(x)
        x = self.denseOut(x)
        return x
    
model = MyLSTM()
model.build(input_shape=(vocab_size, max_length))
model.load_weights('pre-deploy_models/imdbEager.h5')

ResourceExhaustedError: OOM when allocating tensor with shape[125640,500] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node embedding_2/embeddings/Initializer/random_uniform/RandomUniform (defined at <ipython-input-18-83933e43ccb5>:18)  = RandomUniform[T=DT_INT32, _class=["loc:@embedding_2/embeddings/Assign"], dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding_2/embeddings/Initializer/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'embedding_2/embeddings/Initializer/random_uniform/RandomUniform', defined at:
  File "/home/eigenstir/anaconda3/envs/tf_gpu/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/eigenstir/anaconda3/envs/tf_gpu/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_global

In [ ]:
model.predict(test_sentence)

## Ensemble

In [5]:
BATCH_SIZE = 64
LSTM_UNITS = 128
DENSE_UNITS = LSTM_UNITS * 4

def build_model(embedding_matrix):
    words = Input(shape=(max_length,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, 
                           return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, 
                           return_sequences=True))(x)

    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
    hidden = add([hidden, Dense(DENSE_UNITS, activation='relu')(hidden)])
    hidden = Dropout(0.2)(hidden)
    hidden = add([hidden, Dense(DENSE_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    
    model = Model(inputs=words, outputs=result)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

    return model

In [6]:
NUM_MODELS = 2
embedding_matrix = np.loadtxt('embeddings_concat.txt', dtype=int)

def predict_sentence(sentence):
    preds = []
    for model_idx in range(NUM_MODELS):
        model = build_model(embedding_matrix)
        model.load_weights('pre-deploy_models/imdbNonEager{}.h5'.format(str(model_idx)))
        preds = model.predict(sentence)
    return np.average(preds, axis=1)

In [7]:
predict_sentence(test_sentence)

array([0.48127854], dtype=float32)

# Scraper Feed

In [8]:
from urllib.request import urlopen, HTTPError
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint

date_sentiments = {}

for i in range(1,11):
#     page = urlopen('https://www.businesstimes.com.sg/search/crude%2Boil?page='+str(i)).read()
    page = urlopen('https://www.businesstimes.com.sg/search/facebook?page='+str(i)).read()
    soup = BeautifulSoup(page, features="html.parser")
    posts = soup.findAll("div", {"class": "media-body"})
    for post in posts:
        time.sleep(1)
        url = post.a['href']
        date = post.time.text
        print(date, url)
        try:
            link_page = urlopen(url).read()
        except HTTPError as e:
            if e.code == 403:
                continue
            url = url[:-2]
            link_page = urlopen(url).read()
        link_soup = BeautifulSoup(link_page)
        sentences = link_soup.findAll("p")
        passage = ""
        for sentence in sentences:
            passage += sentence.text
        sentiment = predict_sentence(tokenize_and_pad(passage))
        date_sentiments.setdefault(date, []).append(sentiment)

13 Apr 2019 https://www.businesstimes.com.sg/technology/eu-copyright-revamp-targeting-google-facebook-set-for-approval-on-monday
13 Apr 2019 https://www.businesstimes.com.sg/technology/facebook-more-than-doubles-zuckerberg-compensation-to-us226m
12 Apr 2019 https://www.businesstimes.com.sg/technology/facebook-trying-to-stop-its-own-algorithms-from-doing-their-job
10 Apr 2019 https://www.businesstimes.com.sg/technology/facebook-enhances-memorialised-user-accounts
10 Apr 2019 https://www.businesstimes.com.sg/technology/canada-hails-facebook-ban-on-far-right-figures
09 Apr 2019 https://www.businesstimes.com.sg/government-economy/canada-hails-facebook-ban-on-far-right-figures
09 Apr 2019 https://www.businesstimes.com.sg/opinion/facebook-wants-a-faux-regulator-for-internet-speech-it-wont-happen
09 Apr 2019 https://www.businesstimes.com.sg/technology/headway-made-against-election-abuses-facebook
05 Apr 2019 https://www.businesstimes.com.sg/government-economy/facebook-vows-to-block-foreign-ad

InternalError: Dst tensor is not initialized.
	 [[{{node _arg_Placeholder_589_0_0/_5027}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_8__arg_Placeholder_589_0_0", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

In [ ]:
date_sentiment = {}

for k,v in date_sentiments.items():
    date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = round(sum([np.asscalar(o.numpy()) for o in v])/float(len(v)),3)

earliest_date = min(date_sentiment.keys())

print(date_sentiment)

In [ ]:
list(date_sentiment.values())[:5]

# Back trader

In [ ]:
import backtrader as bt
import backtrader.indicators as btind
import datetime
import os.path
import sys

class Sentiment(bt.Indicator):
    lines = ('sentiment',)
    plotinfo = dict(
        plotymargin=0.15,
        plothlines=[0],
        plotyticks=[1.0, 0, -1.0])
    
    def next(self):
        self.date = self.data.datetime
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        self.lines.sentiment[0] = self.sentiment


class SentimentStrat(bt.Strategy):
    params = (
        ('period', 15),
        ('printlog', True),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        # Keep track of pending orders
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)
        self.date = self.data.datetime
        self.sentiment = None
        Sentiment(self.data)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                
            self.bar_executed = len(self)     
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            
        # Write down: no pending order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    
    ### Main Strat ###
    def next(self):
        # log closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        
        # Check if an order is pending. if yes, we cannot send a 2nd one
        if self.order:
            return
        print(self.sentiment)
        # If not in the market and previous sentiment not none
        if not self.position and prev_sentiment:
            # buy if current close more than sma AND sentiment increased by >= 0.5
            if self.dataclose[0] > self.sma[0] and self.sentiment - prev_sentiment >= 0.5:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
        # Already in the market and previous sentiment not none
        elif prev_sentiment:
            # sell if current close less than sma AND sentiment decreased by >= 0.5
            if self.dataclose[0] < self.sma[0] and self.sentiment - prev_sentiment <= -0.5:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.period, self.broker.getvalue()), doprint=True)

In [ ]:
earliest_date

In [ ]:
cerebro = bt.Cerebro()

# Strategy
cerebro.addstrategy(SentimentStrat)

# Data Feed

data = bt.feeds.YahooFinanceData(
        dataname = 'FB',
        fromdate = earliest_date,
        todate = datetime.datetime(2019,4,14),
        reverse = False
    )
cerebro.adddata(data)

cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
cerebro.broker.setcommission(commission=0.001)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.plot()

# With merged data

In [8]:
os.listdir('data/mycollection')

['AMZN_2019-04-16 00:31:56.740133',
 'FB_2019-04-15 23:31:18.494279',
 'MSFT_2019-04-16 00:31:56.740133',
 'AAPL_2019-04-16 00:31:56.740133',
 'FB_2019-04-16 00:21:34.968448',
 'FB_2019-04-16 00:31:56.740133',
 'IBM_2019-04-16 00:31:56.740133',
 'facebook']

In [17]:
li = []

for filename in os.listdir('data/mycollection'):
    df = pd.read_csv('data/mycollection/{}'.format(filename), sep='\t')
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

,Unnamed: 0,0,1,target
0,26,2019-01-22,[SEATTLE] When a chain of physical therapy cen...,0
1,25,2019-01-23,[SAO PAULO] Amazon.com Inc is launching its lo...,0
2,24,2019-01-24,Sao PauloAMAZON.COM Inc is launching its long-...,0
3,20,2019-02-12,[NEW YORK] A lawyer denied on Sunday allegatio...,0
4,19,2019-02-15,San FranciscoAMAZON.COM Inc and General Motors...,0
5,17,2019-02-19,[NEW YORK] Amazon may be pulling the plug on i...,1
6,16,2019-02-21,[NEW YORK] Hundreds of orange robots zoom and ...,0
7,15,2019-02-26,"[JIAXING, China] When Kevin Chiu left his job ...",0
8,14,2019-02-27,[LONDON] Online retailer Amazon will create mo...,1
9,12,2019-03-05,"Santa Monica, United StatesUBER, the rideshari...",1
